# Lesson 1: Multi-Agent Conversation and Stand-up Comedy

Welcome to Lesson 1.

To access the `requirements.txt` file and the`utils` modules, please go to the `File` menu and select`Open...`.

I hope you enjoy this course!

## Setup

In [ ]:
from utils import get_openai_api_key
OPENAI_API_KEY = get_openai_api_key()
llm_config = {"model": "gpt-3.5-turbo"}

## Define an AutoGen agent

In [ ]:
from autogen import ConversableAgent

agent = ConversableAgent( # class
    name="chatbot", # name of the agent
    llm_config=llm_config, # the agent will use the LLM model to generate responses
    human_input_mode="NEVER", # the agent will never seek for human input
    # "ALWAYS" - the agent will always seek for human input before generating a response
)
# Basic set-up for the agent

In [ ]:
# Ask the agent to generate a response to the user's message
reply = agent.generate_reply( # we call the generate_reply method of the agent
    messages=[{"content": "Tell me a joke.", "role": "user"}]
)
print(reply)

In [ ]:
# Even if we asked to repeat the joke, the agent will not repeat it
reply = agent.generate_reply(
    messages=[{"content": "Repeat the joke.", "role": "user"}]
)
print(reply)

## Conversation

Setting up a conversation between two agents, Cathy and Joe, where the memory of their interactions is retained.

If we don't define the system_message the agent will perform as a generic purpose assistant agent. Using the system message we customize the behavior of the agent 

In [ ]:
# First agent
cathy = ConversableAgent(
    name="cathy",
    system_message=
    "Your name is Cathy and you are a stand-up comedian.",
    llm_config=llm_config,
    human_input_mode="NEVER",
)

# Second agent
joe = ConversableAgent(
    name="joe",
    system_message=
    "Your name is Joe and you are a stand-up comedian. "
    "Start the next joke from the punchline of the previous joke.",
    llm_config=llm_config,
    human_input_mode="NEVER",
)

**Note**: You might get a slightly different response (set of jokes) than what is shown in the video

In [ ]:
chat_result = joe.initiate_chat(
    recipient=cathy, 
    message="I'm Joe. Cathy, let's keep the jokes rolling.",
    max_turns=2, # two turns of the conversation and then the chat ends
)

## Print some results

You can print out:

1. Chat history
2. Cost
3. Summary of the conversation

In [ ]:
import pprint

pprint.pprint(chat_result.chat_history)
# we can see the chat history, all the messages exchanged between the agents

In [ ]:
pprint.pprint(chat_result.cost)
# the cost of the chat, expressed in tokens: completion tokens + prompt tokens

In [ ]:
# Summary of the chat
# In this case, the summary is the last message of the chat, but we can also provide a custom summary -> summary_method
pprint.pprint(chat_result.summary)

## Get a better summary of the conversation

In [ ]:
chat_result = joe.initiate_chat(
    cathy, 
    message="I'm Joe. Cathy, let's keep the jokes rolling.", 
    max_turns=2, 
    summary_method="reflection_with_llm",
    summary_prompt="Summarize the conversation",
    # essentially after the conversion ends, we will call a LLM to summarize the conversation
)

In [ ]:
pprint.pprint(chat_result.summary)
# Better summary of the chat, with respect to the last message

## Chat Termination

Chat can be terminated using a termination conditions.

The **termination message** is a bolean function, which takes a message as input and returns a true or false

- Notice that also the system message was changed!

In [ ]:
cathy = ConversableAgent(
    name="cathy",
    system_message=
    "Your name is Cathy and you are a stand-up comedian. "
    "When you're ready to end the conversation, say 'I gotta go'.",
    llm_config=llm_config,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "I gotta go" in msg["content"], # boolean function
    # if "I gotta go" is in the message, the conversation ends
)

joe = ConversableAgent(
    name="joe",
    system_message=
    "Your name is Joe and you are a stand-up comedian. "
    "When you're ready to end the conversation, say 'I gotta go'.",
    llm_config=llm_config,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "I gotta go" in msg["content"] or "Goodbye" in msg["content"],
)
# Both agents will end the conversation if the message contains "I gotta go"

In [ ]:
chat_result = joe.initiate_chat(
    recipient=cathy,
    message="I'm Joe. Cathy, let's keep the jokes rolling."
)

In [ ]:
# cathy is sending a message, to know if joe remembers the last joke
cathy.send(message="What's last joke we talked about?", recipient=joe)